# AI Customer Support Multi-Agent System (RAG + Memory + A2A + API + Long-Running Tasks)
End-to-End Enterprise-Grade Support Agent Built with Gemini, FAISS, Memory Bank & Multi-Agent Orchestration

📝 Project Summary

This project implements a production-grade Customer Support AI Agent using a fully modular multi-agent system, powered by:

🔹 Gemini 2.5 Flash Lite (LLM generation)

🔹 text-embedding-004 (vector embeddings)

🔹 FAISS (semantic retrieval)

🔹 Long-term memory bank (vector memory per user)

🔹 Domain-specialized agents (payment, refund, order, account, technical support)

🔹 A2A protocol (agent-to-agent messages and escalation)

🔹 Orchestrator agent (routing, context compaction, memory + KB fusion)

🔹 FastAPI REST API (full backend server)

🔹 Long-running task engine (pause/resume refund investigation)

🔹 Evaluation suite (intent accuracy, retrieval coverage, hallucination tests, latency)

The system replicates a real customer support backend that can:

Understand user queries

Retrieve knowledge from documents

Use user-specific memories

Route the request to correct domain agent

Escalate queries between agents

Execute long-running background tasks

Serve responses through an API

All within a single Kaggle Notebook, without needing external deployment.

🎯 Problem Statement

Handling customer support tickets—especially at scale—is slow, repetitive, and expensive.
Customers often wait for human agents to respond to questions such as:

“Payment deducted but no confirmation”

“Where is my order?”

“Refund not received yet”

“App keeps crashing”

“How do I delete my account?”

Most of these queries have structured answers, but humans still manually search FAQs or knowledge base articles.

This results in:

High ticket resolution time

Inconsistent answers

Poor customer satisfaction

Bottlenecks during peak seasons

High operational cost

We need an automated, intelligent, reliable support system that can handle 70–80% of tickets without human intervention.

🚀 Solution Overview

This project builds an end-to-end AI Customer Support System using the Multi-Agent architecture taught in the Google Agents Intensive program.

# 2. Environment Setup & Required Installations

In this step, we prepare the Kaggle environment for the multi-agent customer support system.
This cell:

Installs all required Python libraries (Gemini API, FAISS, FastAPI, Sentence Transformers, etc.)

Creates a clean project directory structure inside /kaggle/working

Ensures folders for KB, embeddings, FAISS index, logs, uploads, and models

Confirms successful setup by printing directory paths

This creates a stable foundation for building the Retrieval-Augmented Generation (RAG) pipeline, memory bank, agents, and API server.

In [2]:
# Cell 2 — Setup & Installs
# Run once. On Kaggle this will be fine; on other environments ensure permissions.

# Install required libs (comment/uncomment as needed)
!pip install -q google-generativeai faiss-cpu sentence-transformers fastapi uvicorn nest_asyncio python-multipart

# Imports
import os, time, json, uuid, threading, shutil
from pathlib import Path
from typing import List, Dict, Any
import numpy as np
import pickle
import faiss
from pprint import pprint

# Configure project paths (Kaggle default)
BASE_DIR = Path("/kaggle/working")
DATA_DIR = BASE_DIR / "data"
KB_DIR = DATA_DIR / "knowledge_base"
EMB_DIR = DATA_DIR / "embeddings"
FAISS_DIR = DATA_DIR / "faiss_index"
LOGS_DIR = DATA_DIR / "logs"
UPLOADS_DIR = DATA_DIR / "uploads"
MODELS_DIR = BASE_DIR / "models"

for d in [DATA_DIR, KB_DIR, EMB_DIR, FAISS_DIR, LOGS_DIR, UPLOADS_DIR, MODELS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print("Directories ensured under", DATA_DIR)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
libcugraph-cu12 25.6.0 requires libraft-cu12==25.6.*, but you have libraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 25.6.0 requires pylibraft-cu12==25.6.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 25.6.0 requires rmm-cu12==25.6.*, but you have rmm-cu12 25.2.0 which is incompatible.
Directories ensured under /kaggle/working/data


# 3. Configure Gemini API (LLM + Embeddings)

This cell initializes Google’s Generative AI client (google-generativeai) to enable:

Text generation (via Gemini 2.5 Flash Lite)

Embedding generation (via text-embedding-004)

It securely loads your API key:

First from environment variables

If missing, from Kaggle Secrets under GENAI_API_KEY

Finally, it defines:

EMBED_MODEL = models/text-embedding-004

LLM_MODEL = models/gemini-2.5-flash-lite (low quota usage)

In [3]:
# Cell 3 — Configure Gemini (google-generativeai)
import os
import google.generativeai as genai

# Load API key from Kaggle Secrets
# Make sure you created a secret called: GENAI_API_KEY
GENAI_API_KEY = os.environ.get("GENAI_API_KEY")

if not GENAI_API_KEY:
    try:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        GENAI_API_KEY = user_secrets.get_secret("GENAI_API_KEY")
    except Exception as e:
        raise ValueError("❌ GENAI_API_KEY is missing. Add it in 'Add-ons → Secrets'.") from e

# Configure the model
genai.configure(api_key=GENAI_API_KEY)

# Default Models
EMBED_MODEL = "models/text-embedding-004"
LLM_MODEL = "models/gemini-2.5-flash-lite"

print("Using Embedding model:", EMBED_MODEL)
print("Using LLM model:", LLM_MODEL)
print("🔐 Gemini API Key loaded successfully!")


Using Embedding model: models/text-embedding-004
Using LLM model: models/gemini-2.5-flash-lite
🔐 Gemini API Key loaded successfully!


4. Load Sample Knowledge Base (KB) & Sample Tickets

If this is the notebook's first run, this cell automatically creates:

✔ Sample KB documents

payment_issues.txt

refund_policy.txt

order_status.txt

app_troubleshooting.txt

account_management.txt

These serve as seed knowledge for the RAG system.

✔ Sample support tickets

Stored in tickets_sample.json, used for evaluation and classifier training.

This ensures the full pipeline can run end-to-end even without uploading your own KB.

In [4]:
# Cell 4 — Provide sample KB documents and sample tickets if empty (so notebook runs)
if not any(KB_DIR.glob("*")):
    samples = {
        "payment_issues.txt": """PAYMENT ISSUE TROUBLESHOOTING
1. PAYMENT DEDUCTED BUT NO CONFIRMATION - Wait 5 minutes; if continues, check bank; refund processed automatically on failure.
2. DOUBLE PAYMENT - One payment auto-refunds in 5-7 working days.
3. UPI REFUNDS - 24-72 hours typical; banks vary.""",
        "order_status.txt": """ORDER STATUS INFORMATION
1. Pending - Waiting for payment gateway confirmation.
2. Confirmed - Email/SMS sent.
3. Failed - Refund initiated automatically.""",
        "refund_policy.txt": """REFUND POLICY
Refunds take 5-10 business days for card, 24-72 hours for UPI. Provide Transaction ID to investigate.""",
        "app_troubleshooting.txt": """APP TROUBLESHOOTING
- Clear cache, update app, restart device, reinstall if needed.""",
        "account_management.txt": """ACCOUNT ISSUES
- To delete account contact support with reason. Account deletion is irreversible."""
    }
    for name, text in samples.items():
        p = KB_DIR / name
        p.write_text(text)
    print("Sample KB created in", KB_DIR)

# sample tickets
TICKETS_PATH = DATA_DIR / "tickets_sample.json"
if not TICKETS_PATH.exists():
    sample_tickets = [
        {"id": "t1", "user_message": "Payment deducted but no confirmation"},
        {"id": "t2", "user_message": "Where is my order?"},
        {"id": "t3", "user_message": "I didn't get my refund yet"},
        {"id": "t4", "user_message": "App keeps crashing on open"},
        {"id": "t5", "user_message": "How do I delete my account?"}
    ]
    TICKETS_PATH.write_text(json.dumps(sample_tickets, indent=2))
    print("Sample tickets saved:", TICKETS_PATH)

print("KB files:", list(KB_DIR.glob("*")))
print("Tickets:", TICKETS_PATH)


Sample KB created in /kaggle/working/data/knowledge_base
Sample tickets saved: /kaggle/working/data/tickets_sample.json
KB files: [PosixPath('/kaggle/working/data/knowledge_base/refund_policy.txt'), PosixPath('/kaggle/working/data/knowledge_base/account_management.txt'), PosixPath('/kaggle/working/data/knowledge_base/app_troubleshooting.txt'), PosixPath('/kaggle/working/data/knowledge_base/payment_issues.txt'), PosixPath('/kaggle/working/data/knowledge_base/order_status.txt')]
Tickets: /kaggle/working/data/tickets_sample.json


5. Utility Functions: Cleaning + Chunking

Defines helpful text preprocessing utilities:

🔹 clean_text()

Removes newlines, tabs

Compresses whitespace

🔹 chunk_text()

Splits long documents into small, retrievable chunks (max ~800 chars).

These utilities are crucial for:

1.Building high-quality embeddings

2.Improving RAG retrieval accuracy

In [5]:
# Cell 5 — Utilities
import re
def clean_text(t: str) -> str:
    if not isinstance(t, str):
        return ""
    t = t.replace("\n", " ").replace("\t", " ")
    t = re.sub(r"\s+", " ", t).strip()
    return t

def chunk_text(text: str, max_chars: int = 800):
    parts = [p.strip() for p in text.split("\n\n") if p.strip()]
    out = []
    for p in parts:
        if len(p) <= max_chars:
            out.append(p)
        else:
            for i in range(0, len(p), max_chars):
                out.append(p[i:i+max_chars])
    return out

print("Utilities ready")


Utilities ready


6. Build Text Embeddings for RAG

This step:

Loads and chunks KB documents

Prepares sample tickets

Combines all text passages

Generates embeddings using Gemini:

genai.embed_content() for each text

Saves:

embeddings_gemini.npy

meta_gemini.pkl

If files already exist, they are reloaded.

This forms the core vector database for RAG-based answers.

In [6]:
# Cell 6 — Build embeddings (RAG) and metadata
from tqdm.auto import tqdm

# Load KB docs
kb_docs = []
for p in sorted(KB_DIR.glob("*")):
    if p.suffix.lower() in [".txt", ".md"]:
        kb_docs.append({"name": p.name, "text": p.read_text(encoding="utf-8")})

# Build passages
kb_passages = []
for doc in kb_docs:
    chunks = chunk_text(clean_text(doc["text"]))
    for i, c in enumerate(chunks):
        kb_passages.append({"id": f"{doc['name']}_chunk_{i}", "source": doc['name'], "text": c})

ticket_examples = json.loads(TICKETS_PATH.read_text(encoding="utf-8"))

ticket_texts = [t["user_message"] for t in ticket_examples]
all_texts = [p["text"] for p in kb_passages] + ticket_texts

print(f"KB passages: {len(kb_passages)}, tickets: {len(ticket_examples)}, total texts: {len(all_texts)}")

# Embeddings (use genai.embed_content in batches)
def embed_texts(texts: List[str]) -> np.ndarray:
    embeds = []
    for t in tqdm(texts):
        try:
            resp = genai.embed_content(model=EMBED_MODEL, content=t, task_type="retrieval_document")
            v = np.array(resp["embedding"], dtype="float32")
        except Exception as e:
            print("Embedding error (falling back to random):", e)
            # fallback random vector (not ideal, but keeps notebook running)
            v = np.random.rand(768).astype("float32")
        embeds.append(v)
    return np.vstack(embeds).astype("float32")

emb_path = EMB_DIR / "embeddings_gemini.npy"
meta_path = EMB_DIR / "meta_gemini.pkl"

if not emb_path.exists() or not meta_path.exists():
    embeddings = embed_texts(all_texts)
    np.save(emb_path, embeddings)
    meta = {
        "kb_passages": kb_passages,
        "tickets": ticket_examples,
        "texts": all_texts,
        "kb_count": len(kb_passages),
        "embedding_model": EMBED_MODEL,
        "dim": embeddings.shape[1]
    }
    with open(meta_path, "wb") as f:
        pickle.dump(meta, f)
    print("Saved embeddings and meta")
else:
    embeddings = np.load(emb_path)
    with open(meta_path, "rb") as f:
        meta = pickle.load(f)
    print("Loaded existing embeddings & meta from disk")

print("Embeddings shape:", embeddings.shape)


KB passages: 5, tickets: 5, total texts: 10


  0%|          | 0/10 [00:00<?, ?it/s]

Saved embeddings and meta
Embeddings shape: (10, 768)


7. Create FAISS Vector Index

Here we:

Normalize embeddings with L2 norm

Build a FAISS IndexFlatIP (inner product search)

Save it to disk

Load metadata for retrieval later

This index powers fast semantic search over your knowledge base.

In [7]:
# Cell 7 — Build FAISS index
emb = embeddings
dim = emb.shape[1]
faiss.normalize_L2(emb)

index_path = FAISS_DIR / "kb_faiss_gemini.index"
if not index_path.exists():
    index = faiss.IndexFlatIP(dim)
    index.add(emb)
    faiss.write_index(index, str(index_path))
    shutil.copy(meta_path, FAISS_DIR / "meta_gemini.pkl")
    print("FAISS index built and saved:", index_path)
else:
    index = faiss.read_index(str(index_path))
    print("FAISS index loaded:", index_path)

# load texts
texts = meta.get("texts", [])
print("Number of texts:", len(texts))


FAISS index built and saved: /kaggle/working/data/faiss_index/kb_faiss_gemini.index
Number of texts: 10


8. Define Retriever Functions

This cell enables semantic search:

🔹 embed_query()

Creates an embedding vector for user queries.

🔹 retrieve(query, k)

Returns top-k most relevant KB chunks using FAISS.

This will be used by all domain agents to access contextual knowledge.

In [8]:
# Cell 8 — Retriever (RAG)
def embed_query(query: str):
    try:
        resp = genai.embed_content(model=EMBED_MODEL, content=query, task_type="retrieval_query")
        v = np.array(resp["embedding"], dtype="float32").reshape(1, -1)
    except Exception as e:
        print("Embed query error:", e)
        v = np.random.rand(1, dim).astype("float32")
    faiss.normalize_L2(v)
    return v

def retrieve(query: str, k: int = 4):
    v = embed_query(query)
    try:
        scores, ids = index.search(v, k)
    except Exception as e:
        print("FAISS search error, returning empty:", e)
        return []
    results = []
    for idx, score in zip(ids[0], scores[0]):
        if idx < 0 or idx >= len(texts):
            continue
        results.append({"score": float(score), "text": texts[idx]})
    return results

# quick test
print("Retriever test:", retrieve("payment deducted but no confirmation", 2))


Retriever test: [{'score': 0.7178919315338135, 'text': 'Payment deducted but no confirmation'}, {'score': 0.6268824934959412, 'text': 'PAYMENT ISSUE TROUBLESHOOTING 1. PAYMENT DEDUCTED BUT NO CONFIRMATION - Wait 5 minutes; if continues, check bank; refund processed automatically on failure. 2. DOUBLE PAYMENT - One payment auto-refunds in 5-7 working days. 3. UPI REFUNDS - 24-72 hours typical; banks vary.'}]


9. Long-Term Memory Bank (Vector Store)

Implements a persistent memory system enabling the agent to:

Store summaries of past user tickets

Retrieve memories per user

Maintain context across multiple conversations

Memory stored using:

FAISS index for fast similarity search

memory_meta.pkl for metadata

Functions included:

init_memory_bank()

store_memory()

retrieve_memories()

This satisfies the Sessions & Long-term Memory requirement.

In [9]:
# Cell 9 — Memory Bank (persistent FAISS + metadata)
MEM_DIR = DATA_DIR / "memory_bank"
MEM_DIR.mkdir(parents=True, exist_ok=True)
MEM_INDEX_PATH = MEM_DIR / "memory_faiss.index"
MEM_META_PATH = MEM_DIR / "memory_meta.pkl"

_memory_index = None
_memory_meta = None

def _save_meta():
    with open(MEM_META_PATH, "wb") as f:
        pickle.dump(_memory_meta, f)

def _save_index():
    faiss.write_index(_memory_index, str(MEM_INDEX_PATH))

def init_memory_bank(dim_mb: int = 768):
    global _memory_index, _memory_meta
    if MEM_META_PATH.exists() and MEM_INDEX_PATH.exists():
        with open(MEM_META_PATH, "rb") as f:
            _memory_meta = pickle.load(f)
        _memory_index = faiss.read_index(str(MEM_INDEX_PATH))
        print("Memory Bank loaded:", len(_memory_meta.get("memories", {})), "memories.")
    else:
        from collections import defaultdict
        _memory_meta = {"next_id": 0, "memories": {}, "user_index": defaultdict(list), "dim": dim_mb}
        _memory_index = faiss.IndexFlatIP(dim_mb)
        print("Memory Bank initialized (empty).")
    return True

def _embed_text_mem(text: str):
    try:
        resp = genai.embed_content(model=EMBED_MODEL, content=text, task_type="retrieval_document")
        v = np.array(resp["embedding"], dtype="float32").reshape(1, -1)
    except Exception as e:
        print("Memory embed fallback:", e)
        v = np.random.rand(1, 768).astype("float32")
    faiss.normalize_L2(v)
    return v

def store_memory(user_id: str, text: str, metadata: dict = None):
    global _memory_index, _memory_meta
    if metadata is None:
        metadata = {}
    if _memory_index is None:
        init_memory_bank()
    v = _embed_text_mem(text)
    _memory_index.add(v)
    mem_id = int(_memory_meta["next_id"])
    ts = int(time.time())
    _memory_meta["memories"][mem_id] = {"user_id": user_id, "text": text, "metadata": metadata, "ts": ts}
    _memory_meta["user_index"].setdefault(user_id, []).append(mem_id)
    _memory_meta["next_id"] += 1
    _save_meta()
    _save_index()
    return mem_id

def retrieve_memories(user_id: str, query: str = None, k: int = 5):
    global _memory_index, _memory_meta
    if _memory_index is None:
        init_memory_bank()
    if user_id not in _memory_meta["user_index"] or len(_memory_meta["user_index"][user_id]) == 0:
        return []
    if query is None:
        mem_ids = _memory_meta["user_index"].get(user_id, [])
        mems = sorted([(mid, _memory_meta["memories"][mid]) for mid in mem_ids], key=lambda x: x[1]["ts"], reverse=True)[:k]
        return [{"mem_id": mid, "text": data["text"], "metadata": data["metadata"], "ts": data["ts"], "score": None} for mid, data in mems]
    qv = _embed_text_mem(query)
    D, I = _memory_index.search(qv, min(64, max(1, _memory_meta["next_id"])))
    out = []
    for score, idx in zip(D[0], I[0]):
        if idx < 0:
            continue
        mem_meta = _memory_meta["memories"].get(int(idx))
        if mem_meta and mem_meta["user_id"] == user_id:
            out.append({"mem_id": int(idx), "text": mem_meta["text"], "metadata": mem_meta["metadata"], "ts": mem_meta["ts"], "score": float(score)})
        if len(out) >= k:
            break
    return out

# init memory bank
init_memory_bank()
print("Memory bank ready")


Memory Bank initialized (empty).
Memory bank ready


10. Rule-Based Intent Classifier

A lightweight intent routing system:

Matches keywords against 6 intents:

PAYMENT_ISSUE

ORDER_STATUS

REFUND

ACCOUNT_ISSUE

TECH_SUPPORT

GENERAL_QUERY

Returns:

Detected intent

Assigned agent

Confidence score

This determines which domain agent should process the query.

In [10]:
# Cell 10 — Intent classifier (rules + optional LLM refine)
INTENT_KEYWORDS = {
    "PAYMENT_ISSUE": ["payment", "pay", "deducted", "upi", "refund"],
    "ORDER_STATUS": ["order", "shipping", "delivered", "where is my order"],
    "REFUND": ["refund", "refunded", "money back"],
    "ACCOUNT_ISSUE": ["delete account", "account", "login", "password"],
    "TECH_SUPPORT": ["crash", "error", "app crash", "bug"],
    "GENERAL_QUERY": ["return", "policy", "help", "support"]
}

def route_query(text: str) -> dict:
    t = text.lower()
    for intent, kws in INTENT_KEYWORDS.items():
        for kw in kws:
            if kw in t:
                # Choose agent mapping
                agent_map = {
                    "PAYMENT_ISSUE": "PAYMENT_AGENT",
                    "ORDER_STATUS": "ORDER_AGENT",
                    "REFUND": "REFUND_AGENT",
                    "ACCOUNT_ISSUE": "ACCOUNT_AGENT",
                    "TECH_SUPPORT": "TECH_AGENT",
                    "GENERAL_QUERY": "GENERAL_AGENT"
                }
                return {"intent": intent, "agent": agent_map.get(intent, "GENERAL_AGENT"), "confidence": 0.9, "user_query": text}
    # fallback - label general, low confidence
    return {"intent": "GENERAL_QUERY", "agent": "GENERAL_AGENT", "confidence": 0.3, "user_query": text}

print("route_query running examples:")
print(route_query("Payment deducted but no confirmation"))
print(route_query("How to delete my account?"))


route_query running examples:
{'intent': 'PAYMENT_ISSUE', 'agent': 'PAYMENT_AGENT', 'confidence': 0.9, 'user_query': 'Payment deducted but no confirmation'}
{'intent': 'ACCOUNT_ISSUE', 'agent': 'ACCOUNT_AGENT', 'confidence': 0.9, 'user_query': 'How to delete my account?'}


11. Agent-to-Agent (A2A) Message Helpers

Defines utility wrappers for multi-agent communication:

🔹 a2a_message(target, content)

Requests escalation to another agent.

🔹 final_answer(text)

Marks completion of an agent’s work.

This is part of the A2A protocol required for multi-agent systems.

In [11]:
# Cell 11 — A2A helpers
def a2a_message(target: str, content: str) -> dict:
    return {"type": "a2a_message", "target": target, "content": content}

def final_answer(text: str) -> dict:
    return {"type": "final_answer", "content": text}


12. Domain-Specific Agents (LLM-Powered)

Implements 6 domain agents:

PAYMENT_AGENT

REFUND_AGENT

ORDER_AGENT

TECH_AGENT

ACCOUNT_AGENT

GENERAL_AGENT

Features:

✔ RAG-Augmented LLM

Uses retrieved KB chunks + user memories + the query.

✔ Escalation Logic

E.g., Payment → Refund

✔ Robust fallback answers

In case of LLM quota or errors.

These are the core LLM-powered workers of the system.

In [12]:
# Cell 12 — Domain Agents (use LLM to generate final answers; they may return A2A messages)
def generate_agent_answer(user_query: str, domain: str) -> str:
    # Build context from RAG + memory
    # Ensure retrieve returns list of dicts -> extract texts
    rag_chunks = retrieve(user_query, k=4)
    chunk_texts = [c["text"] if isinstance(c, dict) else c for c in rag_chunks]
    mems = retrieve_memories(user_id="demo_user", query=user_query, k=3)  # placeholder user, real orchestrator will pass user_id
    mem_texts = [m["text"] for m in mems]
    mem_block = "\n".join(f"- {t}" for t in mem_texts) if mem_texts else ""

    context = "\n\n---\n\n".join(chunk_texts)
    prompt = f"""
You are the {domain} of a Customer Support system.
Use ONLY the CONTEXT below to answer, and if insufficient, ask a concise follow-up.

CONTEXT:
{context}

USER QUERY:
{user_query}

USER MEMORIES:
{mem_block}

Respond concisely.
"""
    try:
        model = genai.GenerativeModel(LLM_MODEL)
        resp = model.generate_content(prompt)
        text = resp.text if hasattr(resp, "text") else str(resp)
    except Exception as e:
        print("LLM call failed in generate_agent_answer:", e)
        # fallback: rule-based short answers
        if "payment" in user_query.lower():
            text = "Please wait 5 minutes. If the order fails, refund will be processed automatically."
        elif "refund" in user_query.lower():
            text = "Please provide Transaction ID and screenshot of debit message to investigate."
        elif "order" in user_query.lower():
            text = "Your order is pending payment confirmation; provide order id to check."
        elif "crash" in user_query.lower():
            text = "Try clearing cache, reinstalling the app, and restarting your device."
        elif "delete account" in user_query.lower():
            text = "Account deletion is irreversible. Confirm if you wish to proceed."
        else:
            text = "I do not have enough context; please provide more details."
    return text

# Domain-specific wrappers (may produce A2A if they detect escalation)
def PAYMENT_AGENT(query: str):
    q = query.lower()
    if "refund" in q or "refunded" in q or "didn't receive my refund" in q:
        return a2a_message("REFUND_AGENT", "Payment agent detected refund request.")
    ans = generate_agent_answer(query, "PAYMENT_AGENT")
    return final_answer(ans)

def REFUND_AGENT(query: str):
    ans = generate_agent_answer(query, "REFUND_AGENT")
    return final_answer(ans)

def ORDER_AGENT(query: str):
    ans = generate_agent_answer(query, "ORDER_AGENT")
    return final_answer(ans)

def TECH_AGENT(query: str):
    ans = generate_agent_answer(query, "TECH_AGENT")
    return final_answer(ans)

def ACCOUNT_AGENT(query: str):
    ans = generate_agent_answer(query, "ACCOUNT_AGENT")
    return final_answer(ans)

def GENERAL_AGENT(query: str):
    ans = generate_agent_answer(query, "GENERAL_AGENT")
    return final_answer(ans)

# Agent function map for orchestrator
AGENT_FN_MAP = {
    "PAYMENT_AGENT": PAYMENT_AGENT,
    "REFUND_AGENT": REFUND_AGENT,
    "ORDER_AGENT": ORDER_AGENT,
    "TECH_AGENT": TECH_AGENT,
    "ACCOUNT_AGENT": ACCOUNT_AGENT,
    "GENERAL_AGENT": GENERAL_AGENT,
    # "ESCALATION_AGENT": ... (optional)
}

print("Domain agents defined")


Domain agents defined


13. Multi-Agent Orchestrator (A2A Loop + Memory + Logging)

This is the heart of the system.

The orchestrator:

🔥 Routes query → correct agent
🔥 Executes agent → handles A2A escalations
🔥 Retrieves memories (context engineering)
🔥 Retrieves RAG chunks (compact context)
🔥 Logs full ticket trace
🔥 Stores summary into Memory Bank
🔥 Returns final consolidated response

Supports up to 6 hops of inter-agent messaging.

This fulfills:

Multi-agent system

Context compaction

Memory

A2A protocol

Observability (ticket logs)

In [13]:
# Cell 13 — Orchestrator (A2A loop + memory + compaction + ticket logging)
def compact_context_from_retriever(rag_items: List[Dict[str,Any]]):
    # simple compaction: pick top 4 texts and join
    texts_list = [it["text"] if isinstance(it, dict) else it for it in rag_items][:4]
    return "RELEVANT KNOWLEDGE:\n" + "\n\n---\n\n".join(texts_list)

TICKET_LOG_PATH = LOGS_DIR / "tickets_log.json"
if not TICKET_LOG_PATH.exists():
    TICKET_LOG_PATH.write_text("[]", encoding="utf-8")

def _append_ticket(ticket: dict):
    data = json.loads(TICKET_LOG_PATH.read_text(encoding="utf-8"))
    data.append(ticket)
    TICKET_LOG_PATH.write_text(json.dumps(data, indent=2), encoding="utf-8")

def process_user_query(user_query: str, user_id: str = None, attachments: List[str] = None, debug: bool = False, max_hops: int = 6):
    if attachments is None: attachments = []
    ts = int(time.time())
    user_id = user_id or f"user_{ts}"

    # Route
    routing = route_query(user_query)
    intent = routing.get("intent", "GENERAL_QUERY")
    current_agent = routing.get("agent", "GENERAL_AGENT")
    confidence = routing.get("confidence", 0.0)

    # memory retrieval
    memories = retrieve_memories(user_id, query=user_query, k=3)
    mem_block = "\n".join(f"- {m['text']}" for m in memories) if memories else ""

    # RAG chunks
    rag_chunks = retrieve(user_query, k=4)
    compact_context = compact_context_from_retriever(rag_chunks)

    # build agent input
    agent_input = user_query
    if attachments:
        agent_input += "\n\nAttachments: " + ", ".join(attachments)
    if mem_block:
        agent_input = f"{mem_block}\n\n{agent_input}"

    # A2A loop
    hop = 0
    a2a_trace = []
    last_agent = current_agent
    agent_fn = AGENT_FN_MAP.get(current_agent)
    if agent_fn is None:
        agent_fn = lambda q: final_answer("Sorry, no agent available.")

    final_message = None
    try:
        raw = agent_fn(agent_input)
        # normalize
        if isinstance(raw, dict) and raw.get("type") == "a2a_message":
            msg = raw
        else:
            # ensure dict final_answer
            if isinstance(raw, dict) and raw.get("type")=="final_answer":
                msg = raw
            else:
                msg = final_answer(raw if not isinstance(raw, dict) else str(raw))
        # loop
        while msg.get("type") == "a2a_message" and hop < max_hops:
            hop += 1
            target = msg["target"]
            content = msg["content"]
            a2a_trace.append({"hop": hop, "from": last_agent, "to": target, "content": content})
            target_fn = AGENT_FN_MAP.get(target, lambda q: final_answer("Target agent missing"))
            raw = target_fn(content)
            if isinstance(raw, dict) and raw.get("type")=="a2a_message":
                msg = raw
            else:
                if isinstance(raw, dict) and raw.get("type")=="final_answer":
                    msg = raw
                else:
                    msg = final_answer(raw if not isinstance(raw, dict) else str(raw))
            last_agent = target
        final_message = msg
    except Exception as e:
        if debug: print("Orchestrator agent error:", e)
        final_message = final_answer("Internal orchestration error.")

    final_answer_text = final_message.get("content") if isinstance(final_message, dict) else str(final_message)

    # Summarize (simple summary)
    summary_text = json.dumps({"summary": f"{intent}: {user_query}", "intent": intent, "agent": last_agent})

    # persist ticket
    ticket_id = str(uuid.uuid4())
    ticket_record = {
        "ticket_id": ticket_id,
        "timestamp": ts,
        "user_id": user_id,
        "user_query": user_query,
        "intent": intent,
        "initial_agent": current_agent,
        "final_agent": last_agent,
        "confidence": float(confidence),
        "answer": final_answer_text,
        "summary": summary_text,
        "attachments": attachments,
        "a2a_trace": a2a_trace,
        "compact_context": compact_context
    }
    _append_ticket(ticket_record)

    # store memory (store summary)
    try:
        store_memory(user_id, text=summary_text, metadata={"ticket_id": ticket_id})
    except Exception:
        pass

    return {
        "ticket_id": ticket_id,
        "intent": intent,
        "initial_agent": current_agent,
        "final_agent": last_agent,
        "confidence": float(confidence),
        "answer": final_answer_text,
        "summary": summary_text,
        "a2a_trace": a2a_trace,
        "memory_used": memories,
        "compact_context": compact_context,
        "persisted_to": str(TICKET_LOG_PATH)
    }

print("Orchestrator ready. Try: process_user_query('Payment deducted but no confirmation', user_id='demo')")


Orchestrator ready. Try: process_user_query('Payment deducted but no confirmation', user_id='demo')


14. Long-Running Task Engine (Async Simulation)

Implements a realistic long operation:

Refund investigation (0–100% progress)

Supports:

start_refund_investigation()

get_task_status()

pause_task()

resume_task()

Stored in a JSON-backed task store.

Fulfills pause/resume long-running agents requirement.

In [14]:
# Cell 14 — Long-running Task Manager (simple thread-based)
TASKS_PATH = LOGS_DIR / "long_tasks.json"
if not TASKS_PATH.exists():
    TASKS_PATH.write_text("{}", encoding="utf-8")

_task_store = {}
_task_lock = threading.Lock()

def _save_tasks_to_disk():
    with _task_lock:
        TASKS_PATH.write_text(json.dumps(_task_store, indent=2), encoding="utf-8")

def start_refund_investigation(user_id: str, order_id: str, description: str = ""):
    job_id = str(uuid.uuid4())
    job = {"job_id": job_id, "user_id": user_id, "order_id": order_id, "description": description, "status": "running", "progress": 0, "start_time": int(time.time())}
    _task_store[job_id] = job
    _save_tasks_to_disk()

    def _worker(jid):
        for p in range(0, 101, 10):
            time.sleep(0.8)
            if _task_store[jid]["status"] == "paused":
                while _task_store[jid]["status"] == "paused":
                    time.sleep(0.5)
            _task_store[jid]["progress"] = p
            _save_tasks_to_disk()
        _task_store[jid]["status"] = "done"
        _save_tasks_to_disk()

    th = threading.Thread(target=_worker, args=(job_id,), daemon=True)
    th.start()
    return job

def get_task_status(job_id: str):
    return _task_store.get(job_id)

def pause_task(job_id: str):
    if job_id in _task_store:
        _task_store[job_id]["status"] = "paused"
        _save_tasks_to_disk()
        return _task_store[job_id]
    return None

def resume_task(job_id: str):
    if job_id in _task_store:
        _task_store[job_id]["status"] = "running"
        _save_tasks_to_disk()
        return _task_store[job_id]
    return None

print("Long-running Task engine ready (start_refund_investigation, get_task_status, pause_task, resume_task)")


Long-running Task engine ready (start_refund_investigation, get_task_status, pause_task, resume_task)


15. FastAPI REST API for Multi-Agent System

This cell exposes the entire system over HTTP:

Endpoints:

/ask → Ask main agent

/start_refund_task → Begin long-running job

/task_status/{id} → Get job progress

/pause_task → Pause job

/resume_task → Resume job

Runs uvicorn in background

So FastAPI works inside Kaggle.

This enables:

Client applications

Mobile apps

Frontends

External evaluation

You also satisfy Tooling + Deployment readiness

In [15]:
# Cell 15 — FastAPI REST API (run in notebook; uses thread to run uvicorn)
from fastapi import FastAPI, UploadFile, File, Form
from fastapi.responses import JSONResponse
import nest_asyncio, uvicorn

nest_asyncio.apply()

app = FastAPI(title="Customer Support Multi-Agent API")

@app.post("/ask")
async def ask(user_query: str = Form(...), user_id: str = Form("anonymous"), file: UploadFile = File(None)):
    attachments = []
    if file is not None:
        save_path = UPLOADS_DIR / f"{uuid.uuid4()}_{file.filename}"
        with open(save_path, "wb") as f:
            f.write(await file.read())
        attachments.append(str(save_path))
    # always include nothing else; call orchestrator
    try:
        result = process_user_query(user_query, user_id=user_id, attachments=attachments)
    except Exception as e:
        return JSONResponse({"error": str(e)}, status_code=500)
    return JSONResponse(result)

@app.post("/start_refund_task")
async def start_refund_task(user_id: str = Form(...), order_id: str = Form(...), description: str = Form("")):
    job = start_refund_investigation(user_id=user_id, order_id=order_id, description=description)
    return JSONResponse(job)

@app.get("/task_status/{job_id}")
async def task_status(job_id: str):
    job = get_task_status(job_id)
    if job is None:
        return JSONResponse({"error": "not found"}, status_code=404)
    return JSONResponse(job)

@app.post("/pause_task")
async def pause_job(job_id: str = Form(...)):
    job = pause_task(job_id)
    if job is None:
        return JSONResponse({"error": "not found"}, status_code=404)
    return JSONResponse(job)

@app.post("/resume_task")
async def resume_job(job_id: str = Form(...)):
    job = resume_task(job_id)
    if job is None:
        return JSONResponse({"error": "not found"}, status_code=404)
    return JSONResponse(job)

# Start server in background thread (safe for notebooks)
def _run_uvicorn():
    uvicorn.run(app, host="0.0.0.0", port=8000)

server_thread = threading.Thread(target=_run_uvicorn, daemon=True)
server_thread.start()
print("FastAPI started on port 8000 in background thread. Test with curl -X POST -F 'user_query=Hi' http://127.0.0.1:8000/ask")


FastAPI started on port 8000 in background thread. Test with curl -X POST -F 'user_query=Hi' http://127.0.0.1:8000/ask


16. Quick Sanity Tests

This verifies:

✔ Orchestrator is functioning
✔ RAG retrieval works
✔ Agents produce valid answers
✔ Long-running tasks simulate correctly

Essential for debugging before running evaluation.

In [16]:
# Cell 16 — Quick tests (local)
print("Orchestrator quick test:")
print(process_user_query("Payment deducted but no confirmation", user_id="demo_user"))

# Start a long-running refund task
job = start_refund_investigation("user123", "ORD9988", "Investigating refund for order ORD9988")
print("Started job:", job["job_id"])
time.sleep(1)
print("Status sample:", get_task_status(job["job_id"]))


INFO:     Started server process [47]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Orchestrator quick test:
{'ticket_id': 'e6ce3c0f-f04e-4067-ba44-0515b4e0a430', 'intent': 'PAYMENT_ISSUE', 'initial_agent': 'PAYMENT_AGENT', 'final_agent': 'PAYMENT_AGENT', 'confidence': 0.9, 'answer': "Please wait 5 minutes. If you still haven't received confirmation, check your bank. If the payment failed, a refund will be processed automatically.", 'summary': '{"summary": "PAYMENT_ISSUE: Payment deducted but no confirmation", "intent": "PAYMENT_ISSUE", "agent": "PAYMENT_AGENT"}', 'a2a_trace': [], 'memory_used': [], 'compact_context': "RELEVANT KNOWLEDGE:\nPayment deducted but no confirmation\n\n---\n\nPAYMENT ISSUE TROUBLESHOOTING 1. PAYMENT DEDUCTED BUT NO CONFIRMATION - Wait 5 minutes; if continues, check bank; refund processed automatically on failure. 2. DOUBLE PAYMENT - One payment auto-refunds in 5-7 working days. 3. UPI REFUNDS - 24-72 hours typical; banks vary.\n\n---\n\nORDER STATUS INFORMATION 1. Pending - Waiting for payment gateway confirmation. 2. Confirmed - Email/SMS s

17. Full Agent Evaluation Suite

Runs comprehensive performance scoring:

🧪 Tests include:

Intent classifier accuracy

RAG retriever coverage

Domain agent answer quality

A2A routing correctness

Hallucination safety

End-to-end latency

📊 Produces final report:

Intent Accuracy

RAG Coverage

Agent Quality

A2A correctness

Hallucination Score

E2E Latency

This completes the Agent Evaluation requirement.

In [17]:
# Cell 17 — Evaluation Suite (Intent accuracy, RAG coverage, Agent quality, A2A, Hallucination, E2E Latency)
import time
from statistics import mean

print("📌 Starting Agent Evaluation Suite...\n")

# 1. Intent tests
intent_tests = [
    ("Payment deducted but no confirmation", "PAYMENT_ISSUE"),
    ("I want to delete my account", "ACCOUNT_ISSUE"),
    ("Where is my order?", "ORDER_STATUS"),
    ("App keeps crashing", "TECH_SUPPORT"),
    ("I didn’t receive my refund", "REFUND"),
]
intent_results = []
for text, expected in intent_tests:
    out = route_query(text)
    predicted = out["intent"]
    intent_results.append(predicted == expected)
    print(f"Test: {text} → predicted={predicted}, expected={expected}")
intent_accuracy = sum(intent_results) / len(intent_results)
print("\n🎯 Intent Classifier Accuracy:", intent_accuracy)

# 2. RAG coverage
rag_tests = ["payment deducted", "refund delayed", "order pending", "app crash"]
rag_scores = []
for q in rag_tests:
    chunks = retrieve(q, k=2)
    score = 1 if len(chunks) > 0 else 0
    rag_scores.append(score)
    print(f"RAG for '{q}' → {len(chunks)} chunks")
rag_accuracy = sum(rag_scores) / len(rag_scores)
print("\n🧠 RAG Retriever Coverage:", rag_accuracy)

# 3. Domain agent quality + latency
agent_tests = {
    "PAYMENT_AGENT": "Payment deducted but no confirmation",
    "REFUND_AGENT": "Refund not received after 5 days",
    "ORDER_AGENT": "Where is my order?",
    "TECH_AGENT": "App keeps crashing",
    "ACCOUNT_AGENT": "How to delete my account?",
}
agent_scores = []
latencies = []
for agent, query in agent_tests.items():
    fn = AGENT_FN_MAP[agent]
    t0 = time.time()
    out = fn(query)
    t1 = time.time()
    lat = t1 - t0
    latencies.append(lat)
    # normalize
    if isinstance(out, dict):
        answer_text = out.get("content", "").lower()
    else:
        answer_text = str(out).lower()
    quality = 1 if any(k in answer_text for k in ["refund","order","account","crash","payment"]) else 0
    print(f"\nAgent={agent}, Latency={lat:.2f}s\nOutput = {answer_text[:200]} ...")
    agent_scores.append(quality)
agent_quality = sum(agent_scores) / len(agent_scores)
print("\n🤖 Domain Agent Response Quality:", agent_quality)

# 4. A2A routing
print("\n🛰 Testing A2A routing...")
a2a_out = process_user_query("Payment deducted and refund not received after 5 days", user_id="test_a2a")
a2a_correct = (a2a_out["initial_agent"] == "PAYMENT_AGENT" and a2a_out["final_agent"] == "REFUND_AGENT")
print("A2A chain:", a2a_out["a2a_trace"])
print("A2A Correct:", a2a_correct)

# 5. Hallucination test
print("\n💭 Hallucination Test...")
hallucination_queries = ["Who is the CEO of our company?", "Tell me the interest rate on home loans"]
hall_scores = []
for q in hallucination_queries:
    out = process_user_query(q, user_id="halluc_test")
    ans = out["answer"].lower()
    safe = any(phrase in ans for phrase in ["i do not", "not available", "please provide", "i'm not sure"])
    hall_scores.append(1 if safe else 0)
hall_score = sum(hall_scores) / len(hall_scores)
print("Hallucination score:", hall_score)

# 6. E2E latency
print("\n🚦 Running end-to-end evaluation...")
e2e_queries = ["Payment deducted but no confirmation", "Where is my order?", "Refund not received"]
lat_list = []
for q in e2e_queries:
    t0 = time.time()
    out = process_user_query(q, user_id="eval_e2e")
    lat_list.append(time.time() - t0)
    print(f"Query '{q}' → {lat_list[-1]:.2f}s")
avg_latency = mean(lat_list)
print("\n⚡ Average E2E Latency:", avg_latency)

# Final scorecard
print("\n========================")
print("📊 FINAL AGENT SCORECARD")
print("========================")
print(f"Intent Accuracy:        {intent_accuracy:.2f}")
print(f"Retriever Coverage:     {rag_accuracy:.2f}")
print(f"Agent Quality:          {agent_quality:.2f}")
print(f"A2A Correct:            {a2a_correct}")
print(f"Hallucination Safety:   {hall_score:.2f}")
print(f"E2E Latency:            {avg_latency:.2f} sec")
print("\n✅ Evaluation complete.")


📌 Starting Agent Evaluation Suite...

Test: Payment deducted but no confirmation → predicted=PAYMENT_ISSUE, expected=PAYMENT_ISSUE
Test: I want to delete my account → predicted=ACCOUNT_ISSUE, expected=ACCOUNT_ISSUE
Test: Where is my order? → predicted=ORDER_STATUS, expected=ORDER_STATUS
Test: App keeps crashing → predicted=TECH_SUPPORT, expected=TECH_SUPPORT
Test: I didn’t receive my refund → predicted=PAYMENT_ISSUE, expected=REFUND

🎯 Intent Classifier Accuracy: 0.8
RAG for 'payment deducted' → 2 chunks
RAG for 'refund delayed' → 2 chunks
RAG for 'order pending' → 2 chunks
RAG for 'app crash' → 2 chunks

🧠 RAG Retriever Coverage: 1.0

Agent=PAYMENT_AGENT, Latency=0.86s
Output = please wait 5 minutes. if you still don't have confirmation, please check your bank. a refund will be processed automatically if the payment fails. ...

Agent=REFUND_AGENT, Latency=0.87s
Output = please provide your transaction id so i can investigate this for you. ...

Agent=ORDER_AGENT, Latency=0.72s
Output =